In [ ]:
# import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

plt.rcParams["figure.figsize"] = (20,12)

from matplotlib.lines import Line2D

import matplotlib.dates as mdates
myFmt = mdates.DateFormatter('%H:%M \n%d %b ')
import datetime

In [ ]:
!pip install plotly

In [ ]:
# create function

# calculate metrics
def metrics(true, pred):
    true = true.dropna()
    pred = pred.dropna()
    cm = confusion_matrix(true, pred)
    tn = cm[0][0]
    fp = cm[0][1]
    fn = cm[1][0]
    tp = cm[1][1]

    precision = float(tp/(tp+fp)) 
    recall = float(tp/(tp+fn))  
    f1 = float((2*precision*recall)/(precision+recall))
    tss = float((tp/(tp+fn)) - (fp/(fp+tn)))
    
    return precision, recall, f1, tss, tn, fp, fn, tp

# create conditions
def create_conditions(true, pred):
    return [(true == 1) & (pred == 1),
     (true == 0) & (pred == 0),
     (true == 1) & (pred == 0),
     (true == 0) & (pred == 1)]

# add text to plot
def add_text_plot(lim):
   
    plt.ylabel(r'$\sigma_\phi [rad]$', fontsize=12)
    plt.axhline(0.1, color='silver', linewidth=2)
    plt.grid(True, linewidth=0.3)
    plt.ylim(0, lim)
    plt.gca().xaxis.set_major_formatter(myFmt)

# add legend to plot
def add_legend_cm():
    legend_elements = [Line2D([0], [0], marker='o', color='w', label='TN', markerfacecolor="#1f77b4", markersize=6),
                       Line2D([0], [0], marker='o', color='w', label='FP', markerfacecolor='#ff7f0e', markersize=6),
                       Line2D([0], [0], marker='o', color='w', label='TP', markerfacecolor='#2ca02c', markersize=6),
                       Line2D([0], [0], marker='o', color='w', label='FN', markerfacecolor='#d62728', markersize=6)]
    plt.legend(handles=legend_elements, loc='upper right')

In [ ]:
# select models to compare
model_type_1 = "autoreg15"
model_type_2 = "PC"
shift = 15

In [ ]:
scinti_df_1 = pd.read_csv(f'post_processing/{model_type_1}_predictions_on_test-data_with_time-bin_std-cov_full_test.csv') #upraviť v prípade potreby
scinti_df_2 = pd.read_csv(f'post_processing/{model_type_2}_predictions_on_test-data_with_time-bin_std-cov_full_test.csv')
scinti_df_1 = scinti_df_1.set_index(scinti_df_1["index_time_bin"])
scinti_df_2 = scinti_df_2.set_index(scinti_df_2["index_time_bin"])

In [ ]:
# create one dataframe as intersection
scinti_df = pd.concat([scinti_df_1, scinti_df_2], axis=1)

scinti_df[f"{model_type_1}_time"].fillna(scinti_df[f"{model_type_2}_time"], inplace=True)
scinti_df[f"{model_type_1}_bin"].fillna(scinti_df[f"{model_type_2}_bin"], inplace=True)
scinti_df[f"{model_type_1}_test"].fillna(scinti_df[f"{model_type_2}_test"], inplace=True)
scinti_df.rename(columns={f"{model_type_1}_time":"time_index", f"{model_type_1}_bin": "bin_index",f"{model_type_1}_test":"y_test"}, inplace=True)
scinti_df = scinti_df[["time_index", "bin_index", "index_time_bin", "y_test", 
                       f"{model_type_1}_pred", f"{model_type_1}_cov_test", f"{model_type_1}_cov_pred",
                       f"{model_type_2}_pred", f"{model_type_2}_cov_test", f"{model_type_2}_cov_pred"]]
scinti_df

In [ ]:
# CoV for 1st model
varcoef_true_set1 = 0.5
varcoef_pred_set1 = 0.01
# Cov for 2nd model
varcoef_true_set2 = 0.5
varcoef_pred_set2 = 0.1

scinti_df["True_set1"] = np.where(np.isnan(scinti_df[f"{model_type_1}_cov_test"]), scinti_df[f"{model_type_1}_cov_test"], (scinti_df[f"{model_type_1}_cov_test"] > varcoef_true_set1))
scinti_df["True_set2"] = np.where(np.isnan(scinti_df[f"{model_type_2}_cov_test"]), scinti_df[f"{model_type_2}_cov_test"], (scinti_df[f"{model_type_2}_cov_test"] > varcoef_true_set2))

scinti_df["Pred_set1"] = np.where(np.isnan(scinti_df[f"{model_type_1}_cov_pred"]), scinti_df[f"{model_type_1}_cov_pred"], (scinti_df[f"{model_type_1}_cov_pred"] > varcoef_pred_set1))
scinti_df["Pred_set2"] = np.where(np.isnan(scinti_df[f"{model_type_2}_cov_pred"]), scinti_df[f"{model_type_2}_cov_pred"], (scinti_df[f"{model_type_2}_cov_pred"] > varcoef_pred_set2))

values = [3, 1, 4, 2]

conditions_1 = create_conditions(scinti_df['True_set1'], scinti_df['Pred_set1'])
scinti_df['e1_cm_color'] = np.select(conditions_1, values)
conditions_2 = create_conditions(scinti_df['True_set2'], scinti_df['Pred_set2'])
scinti_df['e2_cm_color'] = np.select(conditions_2, values)
scinti_df['e1_cm_color'] = scinti_df['e1_cm_color'].replace(0,np.nan)
scinti_df['e2_cm_color'] = scinti_df['e2_cm_color'].replace(0,np.nan)

precision1, recall1, f11, tss1, tn1, fp1, fn1, tp1 = metrics(scinti_df["True_set1"], scinti_df["Pred_set1"])
precision2, recall2, f12, tss2, tn2, fp2, fn2, tp2 = metrics(scinti_df["True_set2"], scinti_df["Pred_set2"])

scinti_df = scinti_df.set_index(pd.to_datetime(scinti_df["time_index"]))
bin_used = 13839 #13725
scinti_df = scinti_df[scinti_df["bin_index"]==bin_used]
scinti_df = scinti_df.dropna()

In [ ]:
# plot comparison of predictions
plt.rcParams["figure.figsize"] = (8,6)

size_of_marker = 2.5
size_of_marker2 = 0.5

for i in range(0,365):  
    fig, ax = plt.subplots(2,1)
    plt.setp(ax, xlim=(pd.Timestamp('2019-04-08 12:00:00')+datetime.timedelta(days=i*1), pd.Timestamp('2019-04-08 12:00:00')+datetime.timedelta(days=i*1+1)))
    plt.suptitle(f'Bin {bin_used}', fontsize=14)

    plt.subplot(2, 1, 1).set_title("true test values + CoV test")
    scatter1=plt.scatter(scinti_df.index, scinti_df["y_test"], s=size_of_marker2, c="black", label="True test values")
    scatter8=plt.scatter(scinti_df.index, scinti_df["autoreg15_cov_test"], s=size_of_marker2, c="orange", label="CoV of test")
    plt.legend(markerscale=4)
    plt.axhline(0.1, color='lightgray', linewidth=2, linestyle="--")
    add_text_plot(1)
    
    plt.subplot(2, 1, 2).set_title(f"{model_type_1} prediction + CoV pred", fontsize=12)
    scatter2=plt.scatter(scinti_df.index, scinti_df[f"{model_type_1}_pred"], s=size_of_marker2, c="green", label="Predicted values")
    scatter3=plt.scatter(scinti_df.index, scinti_df[f"{model_type_1}_cov_pred"], s=size_of_marker2, c="orange",  label="CoV of prediction")
    plt.legend(markerscale=4)
    plt.axhline(0.1, color='lightgray', linewidth=2, linestyle="--")
    add_text_plot(0.3)
    plt.xlabel("Time")

    plt.subplots_adjust(left=0.1,
                    bottom=0.1,
                    right=0.9,
                    top=0.9,
                    wspace=0.4,
                    hspace=0.5)
    plt.tight_layout()
    plt.show()


In [ ]:
# plot comparison - predictions (numeric values, classification)

plt.rcParams["figure.figsize"] = (9,13)


size_of_marker=2.5
size_of_marker2 = 0.5


for i in range(0,365):
    fig, ax = plt.subplots(5,1)
    plt.setp(ax, xlim=(pd.Timestamp('2019-02-07 12:00:00')+datetime.timedelta(days=i*1), pd.Timestamp('2019-02-07 12:00:00')+datetime.timedelta(days=i*1+1)))
    plt.suptitle(f'Bin {bin_used}', fontsize=14)

    plt.subplot(5, 1, 1).set_title("True values")
    scatter1=plt.scatter(scinti_df.index, scinti_df["y_test"], s=size_of_marker2, c="black", label="True test values")
    scatter8=plt.scatter(scinti_df.index, scinti_df["autoreg15_cov_test"], s=size_of_marker2, c="orange", label="CoV of test")
    plt.legend(markerscale=4)
    plt.axhline(0.5, color='peachpuff', linewidth=2, linestyle="--")
    add_text_plot(1)

    plt.subplot(5, 1, 2).set_title(f"{model_type_1} prediction + CoV pred", fontsize=12)
    scatter2=plt.scatter(scinti_df.index, scinti_df[f"{model_type_1}_pred"], s=size_of_marker2, c="green", label="prediction")
    scatter3=plt.scatter(scinti_df.index, scinti_df[f"{model_type_1}_cov_pred"], s=size_of_marker2, c="orange",  label="CoV of prediction")
    plt.legend(markerscale=4)
    plt.axhline(0.01, color='peachpuff', linewidth=2, linestyle="--")
    add_text_plot(0.3)

    plt.subplot(5, 1, 3).set_title(f"{model_type_2} prediction + CoV pred", fontsize=12)
    scatter4=plt.scatter(scinti_df.index, scinti_df[f"{model_type_2}_pred"], s=size_of_marker2, c="green",  label="prediction")
    scatter5=plt.scatter(scinti_df.index, scinti_df[f"{model_type_2}_cov_pred"], s=size_of_marker2, c="orange", label="CoV of prediction")
    plt.legend(markerscale=4)
    add_text_plot(0.3)
    plt.axhline(0.1, color='peachpuff', linewidth=2, linestyle="--")

    plt.subplot(5, 1, 4).set_title(f"{model_type_1} - binary classification", fontsize=12)
    scatter6=plt.scatter(scinti_df.index, scinti_df["y_test"]*scinti_df[f"{model_type_1}_pred"].notna(), s=size_of_marker, c=scinti_df['e1_cm_color'], cmap="tab10", vmin=1, vmax=10)
    add_text_plot(1)
    add_legend_cm()

    plt.subplot(5, 1, 5).set_title(f"{model_type_2} - binary classification", fontsize=12)
    scatter7=plt.scatter(scinti_df.index, scinti_df["y_test"]*scinti_df[f"{model_type_2}_pred"].notna(), s=size_of_marker, c=scinti_df['e2_cm_color'], cmap="tab10", vmin=1, vmax=10)
    add_text_plot(1)
    add_legend_cm()


    plt.subplots_adjust(left=0.1,
                        bottom=0.1,
                        right=0.9,
                        top=0.9,
                        wspace=0.4,
                        hspace=0.5)
    plt.tight_layout()
    plt.show()
